<a href="https://colab.research.google.com/github/bhavanasreeps18/CertVerifyMe/blob/main/Indian%20Classical%20Music%20Generation%20using%20LSTM%20and%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install glob2


In [9]:
import glob2

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from music21 import converter, instrument, note, chord


In [22]:
# Load and preprocess MIDI data
def get_notes():
    notes = []
    for file in glob2.glob("indian_classical_music/*.mid"):
        try:
            print("Parsing MIDI file:", file)
            midi = converter.parse(file)
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print("Error parsing MIDI file:", file)
            print("Exception:", e)
    return notes


In [23]:
# Prepare sequences for training
def prepare_sequences(notes, sequence_length):
    sequence_input = []
    sequence_output = []
    pitch_names = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitch_names))
    for i in range(0, len(notes) - sequence_length, 1):
        input_sequence = notes[i:i + sequence_length]
        output_sequence = notes[i + sequence_length]
        sequence_input.append([note_to_int[char] for char in input_sequence])
        sequence_output.append(note_to_int[output_sequence])
    n_patterns = len(sequence_input)
    sequence_input = np.reshape(sequence_input, (n_patterns, sequence_length, 1))
    sequence_input = sequence_input / float(len(pitch_names))
    sequence_output = tf.keras.utils.to_categorical(sequence_output)
    return sequence_input, sequence_output


In [29]:
# Define LSTM model
def create_model(input_shape, output_shape):
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [30]:
# Train the model
def train_model(model, sequence_input, sequence_output, epochs=5, batch_size=64):
    model.fit(sequence_input, sequence_output, epochs=epochs, batch_size=batch_size)


In [31]:
# Generate music using the trained model
def generate_music(model, pitch_names, sequence_length, seed_notes, num_notes):
    int_to_note = dict((number, note) for number, note in enumerate(pitch_names))
    pattern = seed_notes
    generated_notes = []
    for i in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(pitch_names))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        generated_notes.append(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    return generated_notes

In [ ]:
# Main function
if __name__ == '__main__':
    notes = get_notes()
    sequence_length = 100
    sequence_input, sequence_output = prepare_sequences(notes, sequence_length)
    input_shape = (sequence_input.shape[1], sequence_input.shape[2])
    output_shape = sequence_output.shape[1]
    model = create_model(input_shape, output_shape)
    train_model(model, sequence_input, sequence_output)
    seed_notes = sequence_input[np.random.randint(0, len(sequence_input) - 1)]
    generated_notes = generate_music(model, sorted(set(notes)), sequence_length, seed_notes, num_notes=500)


Parsing MIDI file: indian_classical_music/Gaud.mid
Parsing MIDI file: indian_classical_music/Chandra Kauns.mid
Parsing MIDI file: indian_classical_music/bazigar.mid
Parsing MIDI file: indian_classical_music/Durga.mid
Parsing MIDI file: indian_classical_music/Des.mid
Parsing MIDI file: indian_classical_music/Pahadi.mid
Parsing MIDI file: indian_classical_music/Miyan ki Malhar.mid
Parsing MIDI file: indian_classical_music/Tilang.mid
Parsing MIDI file: indian_classical_music/Miaki.mid
Parsing MIDI file: indian_classical_music/Alhiya Bilawal.mid
Parsing MIDI file: indian_classical_music/JaunaPuri.mid
Parsing MIDI file: indian_classical_music/Bhairav.mid
Parsing MIDI file: indian_classical_music/Adana.mid
Parsing MIDI file: indian_classical_music/Tilak Kamode.mid
Parsing MIDI file: indian_classical_music/Peelu.mid
Parsing MIDI file: indian_classical_music/Bahar.mid
Parsing MIDI file: indian_classical_music/DawnOfLove.mid
Parsing MIDI file: indian_classical_music/Behag.mid
Parsing MIDI file: